In [15]:
import peachbox

In [16]:
batch_views = peachbox.CassandraDriver()
batch_views.set_keyspace('batch_views')

In [17]:
query = batch_views.execute("SELECT * FROM reviews")

In [18]:
list(query)

[Row(partition_key=u'547dabac-09d6-11e5-ba55-0242ac110006', product_id=u'B002GHHHI8', review_count=1, review_id=u'-3999312021862635762', score=4, true_as_of_seconds=914112000),
 Row(partition_key=u'1fc91fb2-09d2-11e5-ba55-0242ac110006', product_id=u'B0030MTQNW', review_count=1, review_id=u'-6139567845710774257', score=5, true_as_of_seconds=912556800),
 Row(partition_key=u'00e6a74e-da09-d511-e5ba-550242ac1100', product_id=u'B00005421R', review_count=1, review_id=u'5898966341640611252', score=5, true_as_of_seconds=916272000),
 Row(partition_key=u'07e79538-09d0-11e5-ba55-0242ac110006', product_id=u'6300248550', review_count=1, review_id=u'4554669639422043998', score=5, true_as_of_seconds=894326400),
 Row(partition_key=u'00bf2386-5209-d611-e5ba-550242ac1100', product_id=u'B000I8G5B2', review_count=1, review_id=u'8247119609501692860', score=5, true_as_of_seconds=912297600),
 Row(partition_key=u'0eceb2b4-09d5-11e5-ba55-0242ac110006', product_id=u'B00004CX0D', review_count=1, review_id=u'-225

In [20]:
query = batch_views.execute("SELECT true_as_of_seconds, review_count FROM reviews WHERE product_id='B0002KPI2S'")
review_counts = map(lambda x: (x.true_as_of_seconds, x.review_count), query)
                                                                              

In [21]:
review_counts

[(911520000, 1),
 (911520000, 1),
 (911520000, 1),
 (915580800, 1),
 (911520000, 1),
 (911520000, 1),
 (915580800, 1),
 (915580800, 1),
 (915580800, 1),
 (911520000, 1),
 (911520000, 1),
 (911520000, 1),
 (915580800, 4),
 (911520000, 1),
 (911520000, 1),
 (911520000, 1),
 (915580800, 1),
 (915580800, 1),
 (911520000, 1),
 (915580800, 1),
 (911520000, 1),
 (911520000, 1),
 (911520000, 1),
 (915580800, 1),
 (915580800, 1),
 (911520000, 1),
 (911520000, 1),
 (915580800, 1),
 (911520000, 1),
 (911520000, 1),
 (915580800, 1),
 (911520000, 1),
 (911520000, 1),
 (915580800, 1),
 (911520000, 1),
 (915580800, 1),
 (911520000, 4),
 (911520000, 1),
 (915580800, 1),
 (911520000, 1)]

In [22]:
latest_timestamp = max(map(lambda x: x[0], review_counts))
latest_timestamp

915580800

In [29]:
real_time_views = peachbox.CassandraDriver()
real_time_views.set_keyspace('real_time_views')
real_time_views.execute("SELECT COUNT(*) FROM reviews WHERE product_id='B00004U12X'\
                            AND true_as_of_seconds>"+str(latest_timestamp)+" ALLOW FILTERING")

[Row(count=5)]

In [28]:
import kafka.client                                                                                                        
import kafka.producer      
import ujson as json
import time
                                                                                                                           
client   = kafka.client.KafkaClient('localhost:9092')                                                                      
producer = kafka.producer.SimpleProducer(client)                                                                           
                                                                                                                           
message = {'user_id':'A37I5QIB*9UMPD',
           'product_id':'B00004U12X',
           'review':'review text',
           'summary':'Summarizing a great movie',
           'profile_name':'ItsMe',
           'helpfulness':'2/2',
           'time':int(time.time()),
           'score':5.0}
                                                                                                   
producer.send_messages('movie_reviews', json.dumps(message))                                                                      


[ProduceResponse(topic='movie_reviews', partition=0, error=0, offset=10468)]